# Solving RCPSP with heuristics

We admit that you followed the following [notebook](RCPSP%20%231%20Introduction.ipynb) that introduced you all the basics for RCPSP Problems, in this notebook we will test two greedy heuristics that builds feasible solution.

## Prerequisites

Concerning the python kernel to use for this notebook:
- If running locally, be sure to use an environment with discrete-optimization and minizinc.
- If running on colab, the next cell does it for you.
- If running on binder, the environment should be ready.


In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import importlib
    import os
    import sys  # noqa: avoid having this import removed by pycln

    !{sys.executable} -m pip install -U pip

    # uninstall google protobuf conflicting with ray and sb3
    ! pip uninstall -y protobuf

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install git+https://github.com/airbus/discrete-optimization@master#egg=discrete-optimization

    # be sure to load the proper cffi (downgraded compared to the one initially on colab)
    import cffi

    importlib.reload(cffi)

    # install and configure minizinc
    !curl -o minizinc.AppImage -L https://github.com/MiniZinc/MiniZincIDE/releases/download/2.6.3/MiniZincIDE-2.6.3-x86_64.AppImage
    !chmod +x minizinc.AppImage
    !./minizinc.AppImage --appimage-extract
    os.environ["PATH"] = f"{os.getcwd()}/squashfs-root/usr/bin/:{os.environ['PATH']}"
    os.environ["LD_LIBRARY_PATH"] = (
        f"{os.getcwd()}/squashfs-root/usr/lib/:{os.environ['LD_LIBRARY_PATH']}"
    )

### Imports

In [ ]:
import logging

import nest_asyncio

from discrete_optimization.datasets import fetch_data_from_psplib
from discrete_optimization.rcpsp.rcpsp_model import RCPSPSolution
from discrete_optimization.rcpsp.rcpsp_parser import get_data_available, parse_file
from discrete_optimization.rcpsp.rcpsp_utils import plot_ressource_view, plot_task_gantt
from discrete_optimization.rcpsp.solver.cpm import CPM

# patch asyncio so that applications using async functions can run in jupyter
nest_asyncio.apply()

# set logging level
logging.basicConfig(level=logging.INFO)

### Download datasets

If not yet available, we import the datasets from [psplib](https://www.om-db.wi.tum.de/psplib/data.html).

In [ ]:
needed_datasets = ["j301_1.sm"]
download_needed = False
try:
    files_available_paths = get_data_available()
    for dataset in needed_datasets:
        if len([f for f in files_available_paths if dataset in f]) == 0:
            download_needed = True
            break
except:
    download_needed = True

if download_needed:
    fetch_data_from_psplib()

## First greedy solver : The RCPSP "Pile" solver 
A first idea can be to iteratively build a schedule from source to sink, considering available task at each time, and choosing among the available task with a greedy objective.

A quite natural greedy choice is to use the graph structure of the precedence graph. We consider that task that have a lot of successors state in the graph is more important than the others : indeed it means that doing this task will unlock more following tasks. 

That is what the greedy solver called "Pile" is doing. 

N.B We use the "Pile" name which is the french for "heap", a special structure that we use in the implementation of the solver.

In [ ]:
# Parse some rcpsp file
filepath = [f for f in get_data_available() if "j301_1.sm" in f][0]
rcpsp_problem = parse_file(filepath)

In [ ]:
from discrete_optimization.rcpsp.solver.rcpsp_pile import GreedyChoice, PileSolverRCPSP

In [ ]:
solver = PileSolverRCPSP(problem=rcpsp_problem)
result_storage = solver.solve(greedy_choice=GreedyChoice.MOST_SUCCESSORS)

In [ ]:
best_solution_greedy = result_storage.get_best_solution()
fitnesses = rcpsp_problem.evaluate(best_solution_greedy)
print("fitnesses: ", fitnesses)

In [ ]:
fig_resource_view = plot_ressource_view(
    rcpsp_model=rcpsp_problem,
    rcpsp_sol=best_solution_greedy,
    title_figure="Pile solution",
)
fig_gantt = plot_task_gantt(
    rcpsp_model=rcpsp_problem,
    rcpsp_sol=best_solution_greedy,
)

We observe that the schedule is quite compact, the resource usage seems hard to improve and we are not that far from the theoretical lower bound of 38, that is the optimal duration of the schedule if there was no resource constraint.

## Heuristic based on critical path computation output : SGS

SGS can be seen as a priority based greedy algorithm, the more the task id is on the left side of the permutation, the more it has chance to be scheduled faster. 
We can therefore build heuristic ordering of the task and run SGS on it. One idea it to reuse output of the CPM algorithm to schedule first the task that have the lowest earliest finish date for example, but you can imagine other rules. Let's rerun the ```CPM``` utility already used in previous notebook. 

In [ ]:
CPM??

In [ ]:
solver = CPM(problem=rcpsp_problem)
critical_path = solver.run_classic_cpm()
sol = solver.solve().get_best_solution()
sol.get_start_time(rcpsp_problem.sink_task)

In [ ]:
print("Available fields in CPM output : ", solver.map_node[1].__dict__.keys())

# list sorted by EFD ?
perm_lfd = sorted(rcpsp_problem.tasks_list, key=lambda x: solver.map_node[x]._LFD)
index_perm_lfd = [
    rcpsp_problem.index_task_non_dummy[i]
    for i in perm_lfd
    if i in rcpsp_problem.index_task_non_dummy
]
sol_lfd = RCPSPSolution(problem=rcpsp_problem, rcpsp_permutation=index_perm_lfd)
perm_lsd = sorted(rcpsp_problem.tasks_list, key=lambda x: solver.map_node[x]._LSD)
index_perm_lsd = [
    rcpsp_problem.index_task_non_dummy[i]
    for i in perm_lsd
    if i in rcpsp_problem.index_task_non_dummy
]
sol_lsd = RCPSPSolution(problem=rcpsp_problem, rcpsp_permutation=index_perm_lsd)
# Try different methods ?
# What would be your best results ?
print("LFD ", rcpsp_problem.evaluate(sol_lfd))
print("LSD ", rcpsp_problem.evaluate(sol_lsd))

sol_lsd = RCPSPSolution(problem=rcpsp_problem, rcpsp_permutation=index_perm_lsd)

We let you image better priority rules that could be used, because we are a bit disappointed with the priority list so far :) !